In [ ]:
from OptimizedDataGenerator4 import *
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pydot
import graphviz
from qkeras import QDense, QActivation, quantized_bits, quantized_relu
from plotting import *

noGPU=False
if noGPU:
    tf.config.set_visible_devices([], 'GPU')

print(tf.config.experimental.list_physical_devices())
print(tf.test.is_built_with_cuda())
print(tf.test.is_built_with_gpu_support())
print(tf.test.is_gpu_available())

In [ ]:
#// plotting code from google ai https://www.google.com/search?client=firefox-b-1-d&q=plot+tensorflow+model+history
# then modified
def plotModelHistory(history,modelNum = -999):
    plt.subplot(211)
    # Plot training & validation loss values
    plt.plot(history.history['loss'],label="Train")
    plt.plot(history.history['val_loss'],label="Validation")
    plt.title(f'Model {modelNum} loss and accuracy')
    plt.ylabel('Loss')
    # plt.xlabel('Epoch')
    plt.legend()
    plt.subplot(212)
    # Plot training & validation accuracy values
    plt.plot(history.history['binary_accuracy'],label="Train")
    plt.plot(history.history['val_binary_accuracy'],label="Validation")
    # plt.title(f'Model {modelNum} accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.show()


In [ ]:
validation_dir = "./tf_records1000Daniel/tfrecords_validation/"
train_dir = "./tf_records1000Daniel/tfrecords_train/"
x_feature_description: list = ['x_size','z_global','y_profile','x_profile','cluster', 'y_size', 'x_size', 'y_local']
trainODG = OptimizedDataGenerator(tf_records_dir=train_dir,load_records=True, x_feature_description=x_feature_description)
validationODG = OptimizedDataGenerator(tf_records_dir=validation_dir,load_records=True, x_feature_description=x_feature_description)
# trainODG._parse_tfrecord_fn()
## where do we tell it to use the z-global

In [ ]:

def make_model(total_bits, int_bits):
    """
    Build & compile your QKeras model with the given number of integer bits.
    """
    tf.keras.backend.clear_session()
    # inputs
    input1 = tf.keras.layers.Input(shape=(1,), name="z_global")
    input2 = tf.keras.layers.Input(shape=(1,), name="x_size")
    input3 = tf.keras.layers.Input(shape=(1,), name="y_size")
    input4 = tf.keras.layers.Input(shape=(1,), name="y_local")
    x = tf.keras.layers.Concatenate()([input1, input2, input3, input4])

    ## I want to try this with 1 int bit and 7 fractional
    ## I want to try this with 0 int bit and 7 fractional
    
    # layer 1
    x = QDense(
        17,
        kernel_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        bias_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        #kernel_regularizer=tf.keras.regularizers.L1L2(0.0001),
        ## adds sum of the activations squared to the loss function 
        #activity_regularizer=tf.keras.regularizers.L2(0.0001),
    )(x)
    x = QActivation(
        activation=quantized_relu(total_bits, int_bits),
        name="q_relu1"
    )(x)

    # layer 2 (example—you can tweak per‐layer bits)
    x = QDense(
        20,
        kernel_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        bias_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        #kernel_regularizer=tf.keras.regularizers.L1L2(0.0001),
        ## adds sum of the activations squared to the loss function 
        #activity_regularizer=tf.keras.regularizers.L2(0.0001),
    )(x)
    x = QActivation(
        activation=quantized_relu(total_bits, int_bits),
        name="q_relu2"
    )(x)

    # layer 3
    x = QDense(
        9,
        kernel_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        bias_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        #kernel_regularizer=tf.keras.regularizers.L1L2(0.0001),
        ## adds sum of the activations squared to the loss function 
        #activity_regularizer=tf.keras.regularizers.L2(0.0001),
    )(x)
    x = QActivation(
        activation=quantized_relu(total_bits, int_bits),
        name="q_relu3"
    )(x)

    # layer 4
    x = QDense(
        16,
        kernel_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        bias_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        #kernel_regularizer=tf.keras.regularizers.L1L2(0.0001),
        ## adds sum of the activations squared to the loss function 
        #activity_regularizer=tf.keras.regularizers.L2(0.0001),
    )(x)
    x = QActivation(
        activation=quantized_relu(total_bits, int_bits),
        name="q_relu4"
    )(x)

    # layer 5
    x = QDense(
        8,
        kernel_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        bias_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        #kernel_regularizer=tf.keras.regularizers.L1L2(0.0001),
        ## adds sum of the activations squared to the loss function 
        #activity_regularizer=tf.keras.regularizers.L2(0.0001),
    )(x)
    x = QActivation(
        activation=quantized_relu(total_bits, int_bits),
        name="q_relu5"
    )(x)

    # output
    x = QDense(
        1,
        kernel_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        bias_quantizer=quantized_bits(total_bits, int_bits, alpha=0.001),
        #kernel_regularizer=tf.keras.regularizers.L2(0.0001),
    )(x)
    out = QActivation("smooth_sigmoid")(x)
    
    m = tf.keras.Model(inputs=[input1, input2, input3, input4], outputs=out)
    m.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["binary_accuracy"],
        run_eagerly=True
    )
    return m

models = {}

# 1) define your sweep
total_bits = [16]
int_bits =   [8]

# 2) containers for final metrics
train_losses, val_losses = {}, {}
train_accs,   val_accs   = {}, {}

for i, val in enumerate(total_bits):
    print(f"\n→ training model with {val} total bits and {int_bits[i]} integer bits")
    model = make_model(val, int_bits[i])
    

    name = f"total: {val}, int: {int_bits[i]}"
    models[name] = model

    hist  = model.fit(
        x=trainODG,
        validation_data=validationODG,
        epochs=100        # you can shorten for quick experiments
        #verbose=1
    )
    train_losses[f"int: {int_bits[i]}, fractions: {val - int_bits[i] - 1}"] = hist.history["loss"]
    val_losses[f"int: {int_bits[i]}, fractions: {val - int_bits[i] - 1}"] = hist.history["val_loss"]

    train_accs[f"int: {int_bits[i]}, fractions: {val - int_bits[i] - 1}"] = hist.history["binary_accuracy"]
    val_accs[f"int: {int_bits[i]}, fractions: {val - int_bits[i] - 1}"] = hist.history["val_binary_accuracy"]


plt.subplot(211)
for i, val in enumerate(total_bits):
    plt.plot(train_losses[f"int: {int_bits[i]}, fractions: {val - int_bits[i] - 1}"], label=f"train loss {int_bits[i]}, {val}")
    plt.plot(val_losses[f"int: {int_bits[i]}, fractions: {val - int_bits[i] - 1}"], label=f"val loss {int_bits[i]}, {val}")

plt.legend()
plt.show()



plt.subplot(212)
# … your plotting calls …
for i, val in enumerate(total_bits):
    plt.plot(train_accs[f"int: {int_bits[i]}, fractions: {val - int_bits[i] - 1}"], label=f"train acc {int_bits[i]}, {val}")
    plt.plot(val_accs[f"int: {int_bits[i]}, fractions: {val - int_bits[i] - 1}"], label=f"val acc {int_bits[i]}, {val}")


# place the legend to the right, outside the plot
plt.legend(
    loc="upper left",
    bbox_to_anchor=(1.02, 1),      # x=1.02 = just to the right of the axes, y=1 = top
    borderaxespad=0                # no padding between axes and legend
)
plt.tight_layout()    

plt.show()






In [ ]:

graph = {}
print(len(models))
for name, m in models.items():
    loss, acc = m.evaluate(validationODG)
    graph[name] = acc
    print(acc)

names = list(graph.keys())
accuracies = list(graph.values())

plt.figure(figsize=(10, 6))
bars = plt.bar(names, accuracies)

# Add accuracy labels on top of each bar
for bar, acc in zip(bars, accuracies):
    plt.text(
        bar.get_x() + bar.get_width() / 2,   # x-position (center of bar)
        bar.get_height(),                    # y-position (top of bar)
        f"{acc:.3f}",                        # formatted accuracy
        ha='center', va='bottom'             # align center, place just above bar
    )

plt.xlabel('Model (total_bits_int_bits)')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy per Quantized Model')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:

plot_weights(models, per_layer=False, mode="stem", include_bias=True)

